##instal libraries we need

In [1]:
import pandas as pd

In [2]:
!pip install transformers

In [3]:
import transformers

In [4]:
!pip install datasets transformers


#### load the dataset from hugging face's website

In [5]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [6]:
# we take the columns we need for our model and prepared them
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 3019
})

###Explode Comments and Create a Dataset with One Comment per Row

In [7]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]

In [8]:
# we have two comments here
df["comments"][1].tolist()

['There is a speed up in Windows machines:\r\n- From `13m 52s` to `11m 10s`\r\n\r\nIn Linux machines, some workers crash with error message:\r\n```\r\nOSError: [Errno 12] Cannot allocate memory\r\n```',
 'There is also a speed up in Linux machines:\r\n- From `7m 30s` to `5m 32s`']

In [9]:
# we use explode() to sperate each comment (one comment per row)
# and 'title' values are duplicated accordingly
comments_df = df.explode("comments", ignore_index=True)
comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/pull/2955,Update legacy Python image for CI tests in Linux,NaN,"Instead of legacy, use next-generation conveni..."
1,https://github.com/huggingface/datasets/pull/2954,Run tests in parallel,There is a speed up in Windows machines:\r\n- ...,Run CI tests in parallel to speed up the test ...
2,https://github.com/huggingface/datasets/pull/2954,Run tests in parallel,There is also a speed up in Linux machines:\r\...,Run CI tests in parallel to speed up the test ...
3,https://github.com/huggingface/datasets/pull/2952,Fix missing conda deps,NaN,`aiohttp` was added as a dependency in #2662 b...


In [10]:
# count num of comments
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 7238
})

### Filter Short Comments and Concatenate Text

In [11]:
print(comments_dataset.column_names)

['html_url', 'title', 'comments', 'body']


In [12]:
# first check if the comment is exist and then calculate the length
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split()) if x["comments"] else 0}
)
# lambda arguments : expression

Map:   0%|          | 0/7238 [00:00<?, ? examples/s]

In [13]:
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
comments_dataset

Filter:   0%|          | 0/7238 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 4071
})

In [14]:
def concatenate_text(examples):
  text = ""
  if examples["title"]:
    text += examples["title"]
  text += " \n "
  if examples["body"]:
    text += examples["body"]
  text += " \n "
  if examples["comments"]:
    text += examples["comments"]
  return {"text": text}

In [15]:
comments_dataset = comments_dataset.map(concatenate_text)

Map:   0%|          | 0/4071 [00:00<?, ? examples/s]

###Create Text Embeddings

In [16]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
print(comments_dataset.column_names)

['html_url', 'title', 'comments', 'body', 'comment_length', 'text']


In [18]:
import torch

device = torch.device("cuda")  # Utilize GPU if available
model.to(device)

def get_embeddings(text_list):
  encoded_input = tokenizer(
      text_list, padding=True, truncation=True, return_tensors="pt"
  )
  encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
  model_output = model(**encoded_input)
  return cls_pooling(model_output)

def cls_pooling(model_output):
  return model_output.last_hidden_state[:, 0]


In [19]:
# Helper function to get embeddings
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [20]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [21]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [22]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/4071 [00:00<?, ? examples/s]

In [23]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [24]:
!pip install torch torchvision torchtext --extra-index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [25]:


!pip install faiss-gpu --upgrade

embeddings_dataset = embeddings_dataset.add_faiss_index("embeddings")

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

  0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
!pip show faiss-gpu

Name: faiss-gpu
Version: 1.7.2
Summary: A library for efficient similarity search and clustering of dense vectors.
Home-page: https://github.com/kyamagu/faiss-wheels
Author: Kota Yamaguchi
Author-email: KotaYamaguchi1984@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [27]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [28]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?
SCORE: 25.505037307739258
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: The local dataset builders (csv, text , json and pandas) are now part of the `datasets` package since #1726 :)
You can now use them offline
```python
datasets = load_dataset('text', data_files=data_files)
```

We'll do a new release soon
SCORE: 24.5555477142334
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: I opened a PR that allows to reload modules that have already been loaded once even if there's no internet